In [118]:
import numpy as np 
import yfinance as yf
import matplotlib.pyplot as plt 
import pandas as pd 
#import backtesting as bt
from backtesting import Strategy
from backtesting.lib import crossover
from backtesting import Backtest
import pandas_ta as ta
# Example OHLC daily data for Google Inc.
from backtesting.test import GOOG

from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split

import tensorflow as tf

In [119]:
# Strategy

def SMA(values, n):
    """
    Return simple moving average of `values`, at
    each step taking into account `n` previous values.
    """
    return pd.Series(values).rolling(n).mean()

def BOLL(values, n, m):
    """
    Return Bollinger Bands for `values`, at
    each step taking into account `n` previous values
    and `m` standard deviations away from the mean.
    """
    sma = SMA(values, n)
    std = pd.Series(values).rolling(n).std()
    return sma, sma + m * std, sma - m * std

def Dochian(data, high_length, low_length):
    """
    Return Dochian Channel for `values`, at
    each step taking into account `n` previous values
    """
    return ta.donchian(data.High.s, data.Low.s, low_length, high_length, offset=1, talib=False)

def ATR(data, n):
    return np.array(ta.atr(high=data.High.s, low=data.Low.s, close=data.Close.s, length=n, talib=False))
    
    # tr = pd.Series(np.maximum(high - low, high - close, close - low))
    # return tr.rolling(n).mean()\

class turtleTrading(Strategy):
    upper_legnth = 20
    lower_length = 10
    atr_length = 20
    stop_loss_ratio = 11
    portion_ratio = 0.02
    
    def init(self):    
        #print(ATR(self.data, 20))
        self.atr = self.I(ATR, self.data, self.atr_length)
        self.donchian = self.I(Dochian, self.data, self.upper_legnth, self.lower_length)
        self.last_buy_price = 0
        self.last_buy_atr = 0
        
    def next(self):
        # If today's close exceed the donchian_upper_bound, buy, record price and ATR
        if crossover(self.data.Close, self.donchian[2]):
            portion = min((self.portion_ratio*self.data.Close[-1])/self.atr[-1], 1)
            self.buy(size=portion)
            self.last_buy_price = self.data.Close[-1]
            self.last_buy_atr = self.atr[-1]
        # If today's close is below the donchian_lower_bound, sell (end trade)
        elif crossover(self.donchian[0], self.data.Close):
            self.position.close() 
        # If price decresed by twice N at the last buy point, sell all (stop loss)
        elif self.position and self.data.Low[-1] < (self.last_buy_price - self.stop_loss_ratio * self.last_buy_atr):
            self.position.close()
            
stock_code = "ABBV" 
data_collecting_begaining_date = "2020-01-01"
data_collecting_ending_date = "2024-10-20"
period_of_collecting = "1d" 

def stock_price_collecting(code, datestart, dateend, period):
    data = yf.Ticker(code)
    prices = data.history(start=datestart, end=dateend, interval=period)
    return prices

data = stock_price_collecting(stock_code, data_collecting_begaining_date, data_collecting_ending_date, period_of_collecting)
data = pd.DataFrame(data)
train_data = data['2020-01-01':'2023-01-01']
test_data = data['2023-01-02':'2024-10-20']

bt = Backtest(train_data, turtleTrading, cash=10000, commission=.002)
stats = bt.optimize(
    upper_legnth=range(10, 40, 10),
    lower_length=range(5, 20, 5),
    atr_length=range(10, 40, 10),
    stop_loss_ratio=range(1, 5, 1),
    portion_ratio= [x/100 for x in range(1,5,1)],
    constraint=lambda p: p.upper_legnth > p.lower_length,
    maximize="Return [%]"
)
#stats = bt.run()

# Print optimized parameters
print("Optimized Parameters:")
print(f"Upper Length: {stats._strategy.upper_legnth}")
print(f"Lower Length: {stats._strategy.lower_length}")
print(f"ATR Length: {stats._strategy.atr_length}")
print(f"Stop Loss Ratio: {stats._strategy.stop_loss_ratio}")
print(f"Portion Ratio: {stats._strategy.portion_ratio}")

print(stats)
bt.plot()

/opt/anaconda3/envs/wghs2/lib/python3.12/site-packages/backtesting/backtesting.py:1539: UserWarning: Searching for best of 336 configurations.
  output = _optimize_grid()
/opt/anaconda3/envs/wghs2/lib/python3.12/site-packages/backtesting/backtesting.py:1426: UserWarning: For multiprocessing support in `Backtest.optimize()` set multiprocessing start method to 'fork'.
  warnings.warn("For multiprocessing support in `Backtest.optimize()` "


Optimized Parameters:
Upper Length: 20
Lower Length: 15
ATR Length: 30
Stop Loss Ratio: 4
Portion Ratio: 0.01
Start                     2020-01-02 00:00...
End                       2022-12-30 00:00...
Duration                   1093 days 00:00:00
Exposure Time [%]                   56.878307
Equity Final [$]                 15502.732549
Equity Peak [$]                  16945.179917
Return [%]                          55.027325
Buy & Hold Return [%]              107.371443
Return (Ann.) [%]                   15.736253
Volatility (Ann.) [%]               14.648106
Sharpe Ratio                         1.074286
Sortino Ratio                         1.74289
Calmar Ratio                         1.003807
Max. Drawdown [%]                  -15.676578
Avg. Drawdown [%]                   -2.127507
Max. Drawdown Duration      323 days 00:00:00
Avg. Drawdown Duration       29 days 00:00:00
# Trades                                   38
Win Rate [%]                        60.526316
Best Trade [%]  

/opt/anaconda3/envs/wghs2/lib/python3.12/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


GridPlot(id='p22743', ...)

In [116]:
turtleTrading.upper_legnth = stats._strategy.upper_legnth
turtleTrading.lower_length = stats._strategy.lower_length
turtleTrading.atr_length = stats._strategy.atr_length
turtleTrading.stop_loss_ratio = stats._strategy.stop_loss_ratio
turtleTrading.portion_ratio = stats._strategy.portion_ratio

bt_test = Backtest(test_data, turtleTrading, cash=10000)
stats_test = bt_test.run()
print(stats_test)
bt_test.plot()

Start                     2023-01-03 00:00...
End                       2024-10-18 00:00...
Duration                    653 days 23:00:00
Exposure Time [%]                   54.424779
Equity Final [$]                 10408.299749
Equity Peak [$]                  10948.948937
Return [%]                           4.082997
Buy & Hold Return [%]               25.429404
Return (Ann.) [%]                    2.256193
Volatility (Ann.) [%]               12.462942
Sharpe Ratio                         0.181032
Sortino Ratio                        0.230654
Calmar Ratio                         0.166962
Max. Drawdown [%]                  -13.513232
Avg. Drawdown [%]                   -3.790944
Max. Drawdown Duration      284 days 00:00:00
Avg. Drawdown Duration       59 days 00:00:00
# Trades                                   23
Win Rate [%]                        34.782609
Best Trade [%]                      17.691428
Worst Trade [%]                    -10.217829
Avg. Trade [%]                    

/opt/anaconda3/envs/wghs2/lib/python3.12/site-packages/backtesting/_plotting.py:456: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  .resample(resample_rule, label='left')


GridPlot(id='p22316', ...)

In [117]:



def objective(params): #不是数学函数，不可微分

    upper_length, lower_length, atr_length, stop_loss_ratio, portion_ratio = params
    turtleTrading.upper_length = int(upper_length)
    turtleTrading.lower_length = int(lower_length)
    turtleTrading.atr_length = int(atr_length)
    turtleTrading.stop_loss_ratio = stop_loss_ratio
    turtleTrading.portion_ratio = portion_ratio

    bt = Backtest(train_data, turtleTrading, cash=10000)
    stats = bt.run()
    return tf.convert_to_tensor(-stats['Return [%]'], dtype=tf.float32)
# bt = Backtest(historical_price_data, turtleTrading, cash=1_000_000)
# stats = bt.run()
# print(stats)
# bt.plot()

params = tf.Variable([20.0, 10.0, 20.0, 2.0, 0.1], dtype=tf.float32)
optimizer = tf.optimizers.Adam(learning_rate=0.0001)

# Train the model
for step in range(1000):
    with tf.GradientTape() as tape:
        tape.watch(params)  # Ensure params are being watched
        loss = objective(params)

    grads = tape.gradient(loss, [params])
    print(grads)
    optimizer.apply_gradients(zip(grads, [params]))
    if step % 100 == 0:
        print(f'Step {step}, Loss: {loss.numpy()}, Params: {params.numpy()}')

turtleTrading.upper_length = int(params[0].numpy())
turtleTrading.lower_length = int(params[1].numpy())
turtleTrading.atr_length = int(params[2].numpy())
turtleTrading.stop_loss_ratio = params[3].numpy()
turtleTrading.portion_ratio = params[4].numpy()

[None]


ValueError: No gradients provided for any variable.

In [21]:
import unittest
import tensorflow as tf
#from your_module import objective  # Replace 'your_module' with the actual module name

class TestTurtleTrading(unittest.TestCase):
    def setUp(self):
        self.params = tf.Variable([20.0, 10.0, 20.0, 2.0, 0.1], dtype=tf.float32)
        self.optimizer = tf.optimizers.Adam(learning_rate=0.0001)

    def test_gradient_descent(self):
        for step in range(1000):
            with tf.GradientTape() as tape:
                loss = objective(self.params)
            grads = tape.gradient(loss, [self.params])
            self.optimizer.apply_gradients(zip(grads, [self.params]))
            if step % 100 == 0:
                print(f'Step {step}, Loss: {loss.numpy()}, Params: {self.params.numpy()}')
                self.assertIsNotNone(loss.numpy())
                self.assertEqual(len(self.params.numpy()), 5)

if __name__ == '__main__':
    unittest.main(argv=[''], exit=False)

E
ERROR: test_gradient_descent (__main__.TestTurtleTrading.test_gradient_descent)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "/var/folders/2q/hj1hhl956zlcpt20zgdynvg40000gn/T/ipykernel_27174/3575116118.py", line 15, in test_gradient_descent
    self.optimizer.apply_gradients(zip(grads, [self.params]))
  File "/opt/anaconda3/envs/wghs2/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py", line 344, in apply_gradients
    self.apply(grads, trainable_variables)
  File "/opt/anaconda3/envs/wghs2/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py", line 397, in apply
    grads, trainable_variables = self._filter_empty_gradients(
                                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/envs/wghs2/lib/python3.12/site-packages/keras/src/optimizers/base_optimizer.py", line 729, in _filter_empty_gradients
    raise ValueError("No gradients provided for any variable.

In [ ]:
# Backtesting

stock_code = "GE"
